# Clustering Crypto

In [248]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [249]:
# Load the crypto_data.csv dataset.
file_path = 'Resources\crypto_data.csv' 
Crypto_df = pd.read_csv(file_path, index_col=0)
Crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [250]:
Crypto_df.columns

Index(['CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined',
       'TotalCoinSupply'],
      dtype='object')

In [251]:
Crypto_df.IsTrading.unique()

array([ True, False])

In [252]:
# Keep all the cryptocurrencies that are being traded.
Crypto_df = Crypto_df.loc[Crypto_df['IsTrading'] == True]

In [253]:
Crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [254]:
Crypto_df.Algorithm.unique()

array(['Scrypt', 'X11', 'SHA-256', 'X13', 'Ethash', 'CryptoNight-V7',
       'Equihash', 'SHA-512', 'Multiple', 'X15', 'NIST5', 'Quark',
       'Groestl', 'PoS', 'NeoScrypt', 'SHA3', 'HybridScryptHash256',
       'Scrypt-n', 'PHI1612', 'Lyra2REv2', 'CryptoNight', 'Shabal256',
       'Counterparty', 'Blake', 'Momentum', 'Stanford Folding', 'QuBit',
       'XG Hash', 'M7 POW', 'Curve25519', 'Lyra2RE', 'QUAIT', 'vDPOS',
       'Blake2b', 'BLAKE256', '1GB AES Pattern Search', 'Dagger',
       'CryptoNight-Lite', 'X11GOST', 'SHA-256D', 'POS 3.0',
       'Progressive-n', 'DPoS', 'Lyra2Z', 'X14', 'Time Travel', 'Argon2',
       'Keccak', 'Blake2S', 'Dagger-Hashimoto', '536', 'Argon2d',
       'Cloverhash', 'Skein', 'SkunkHash v2 Raptor',
       'VeChainThor Authority', 'Ouroboros', 'POS 2.0', 'SkunkHash',
       'C11', 'Proof-of-BibleHash', 'SHA-256 + Hive',
       'Proof-of-Authority', 'XEVAN', 'VBFT', 'YescryptR16', 'IMesh',
       'Green Protocol', 'Semux BFT consensus', 'X16R', 'Tribus',


In [255]:
# Keep all the cryptocurrencies that have a working algorithm.
Crypto_df = Crypto_df.dropna(axis=0, subset=['Algorithm'])

In [256]:
# Remove the "IsTrading" column. 
Crypto_df = Crypto_df.drop(['IsTrading'],axis = 1)

In [257]:
Crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [258]:
# Remove rows that have at least 1 null value.
Crypto_df = Crypto_df.dropna()

In [259]:
# Keep the rows where coins are mined.
Crypto_df = Crypto_df.loc[Crypto_df['TotalCoinsMined'] > 0]
Crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [260]:
# Create a new DataFrame that holds only the cryptocurrencies names.
Names_df = Crypto_df[["CoinName"]]

In [261]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
Crypto_df = Crypto_df.drop(['CoinName'], axis=1)
Crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [262]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(Crypto_df, columns=['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [263]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [264]:
# Using PCA to reduce dimension to three principal components.
pca_reduce = PCA(n_components=3)

In [265]:
# Create a DataFrame with the three principal components.
x_PCA = pca_reduce.fit_transform(X_scaled)
x_PCA

array([[-0.34368485,  1.11761827, -0.47520678],
       [-0.32700377,  1.11767052, -0.47572741],
       [ 2.29995391,  1.68644642, -0.6205224 ],
       ...,
       [ 0.33660574, -2.31310133,  0.33792631],
       [-0.15453488, -1.98714728,  0.30427143],
       [-0.28213599,  0.79145682, -0.29934424]])

In [266]:
index_values = (X.index.tolist())
index_values

['42',
 '404',
 '1337',
 'BTC',
 'ETH',
 'LTC',
 'DASH',
 'XMR',
 'ETC',
 'ZEC',
 'BTS',
 'DGB',
 'BTCD',
 'XPY',
 'PRC',
 'KOBO',
 'SPR',
 'ARG',
 'AUR',
 'BLU',
 'XMY',
 'MOON',
 'ZET',
 'SXC',
 'QTL',
 'ENRG',
 'QRK',
 'RIC',
 'DGC',
 'BTB',
 'CAT',
 'CBX',
 'CCN',
 'CRYPT',
 'CSC',
 'DMD',
 'XVG',
 'DVC',
 'EAC',
 'EFL',
 'EMC2',
 'EMD',
 'EXCL',
 'FLT',
 'FRK',
 'FTC',
 'GDC',
 'GLC',
 'GLD',
 'HBN',
 'HYP',
 'IFC',
 'IOC',
 'IXC',
 'KGC',
 'LKY',
 'LTB',
 'MAX',
 'MEC',
 'MED',
 'MINT',
 'MINC',
 'MZC',
 'NAUT',
 'NAV',
 'NOBL',
 'NMC',
 'NYAN',
 'OPAL',
 'ORB',
 'POT',
 'PXC',
 'RDD',
 'RPC',
 'SBC',
 'SMC',
 'SUPER',
 'SYNC',
 'SYS',
 'TES',
 'TGC',
 'TIT',
 'TOR',
 'TRC',
 'UNB',
 'UNO',
 'URO',
 'USDE',
 'UTC',
 'VIA',
 'VRC',
 'VTC',
 'WDC',
 'XC',
 'XCR',
 'XJO',
 'XST',
 'ZCC',
 'BCN',
 'XDN',
 'BURST',
 'SJCX',
 'MONA',
 'NTRN',
 'FAIR',
 'NLG',
 'RBY',
 'PTC',
 'KORE',
 'WBB',
 'NOTE',
 'FLO',
 '8BIT',
 'STV',
 'ABY',
 'FLDC',
 'U',
 'UIS',
 'CYP',
 'OMC',
 'VTR',
 'GRE'

In [267]:
pcs_df = pd.DataFrame(data = x_PCA, columns=["PC 1", 
                                             "PC 2", 
                                             "PC 3"], index = index_values)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.343685,1.117618,-0.475207
404,-0.327004,1.117671,-0.475727
1337,2.299954,1.686446,-0.620522
BTC,-0.142658,-1.340253,0.143713
ETH,-0.148724,-2.075908,0.312357
LTC,-0.171513,-1.076819,0.021344
DASH,-0.393035,1.239079,-0.455469
XMR,-0.150617,-2.253049,0.379688
ETC,-0.147163,-2.076015,0.312327
ZEC,-0.154534,-1.987147,0.304271


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [268]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

Running K-Means with `k=4`

In [269]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
print(predictions)
pcs_df["Class"] = model.labels_

[0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1
 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1
 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0
 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0
 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1
 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1
 0 1 0 1 0 0 3 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 1 0
 1 1 0 1 1 1 1 0 0 3 1 1 1 0 3 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1
 0 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 0 1 0 0 1 0 0 1 3 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 1 0 

In [270]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = Crypto_df.join(pcs_df, how='inner')
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343685,1.117618,-0.475207,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.327004,1.117671,-0.475727,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.299954,1.686446,-0.620522,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.142658,-1.340253,0.143713,1
ETH,Ethash,PoW,1.076842e+08,0,-0.148724,-2.075908,0.312357,1


In [271]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(Names_df, how='inner')
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343685,1.117618,-0.475207,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.327004,1.117671,-0.475727,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.299954,1.686446,-0.620522,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.142658,-1.340253,0.143713,1,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.148724,-2.075908,0.312357,1,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.171513,-1.076819,0.021344,1,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.393035,1.239079,-0.455469,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150617,-2.253049,0.379688,1,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.147163,-2.076015,0.312327,1,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.154534,-1.987147,0.304271,1,ZCash


In [272]:
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.343685,1.117618,-0.475207,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.327004,1.117671,-0.475727,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.299954,1.686446,-0.620522,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.142658,-1.340253,0.143713,1,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.148724,-2.075908,0.312357,1,Ethereum
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.171513,-1.076819,0.021344,1,Litecoin
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.393035,1.239079,-0.455469,0,Dash
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150617,-2.253049,0.379688,1,Monero
ETC,Ethash,PoW,1.133597e+08,210000000,-0.147163,-2.076015,0.312327,1,Ethereum Classic
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.154534,-1.987147,0.304271,1,ZCash


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [273]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df, 
    x="PC 1", 
    y="PC 2", 
    z="PC 3", 
    color="Class", 
    symbol="Class", 
    hover_name="CoinName", 
    hover_data=["Algorithm", "TotalCoinsMined", "TotalCoinSupply"])
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [274]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 
                                   'Algorithm', 
                                   'ProofType', 
                                   'TotalCoinsMined', 
                                   'TotalCoinSupply', 
                                   'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Class]

In [275]:
# Print the total number of tradable cryptocurrencies.
clustered_df['CoinName'].count()

532

In [276]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
cluster_df = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]
X_minmax = MinMaxScaler().fit_transform(cluster_df)
X_minmax

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [277]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
index_values = (clustered_df.index.tolist())
plot_df = pd.DataFrame(
    data = X_minmax, columns=["TotalCoinSupply_scaled", 
                              "TotalCoinsMined_scaled"], index = index_values)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = plot_df.join(Names_df, how='inner')

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
class_df = clustered_df['Class']
plot_df = plot_df.join(class_df, how='inner')

plot_df.head(10)

,TotalCoinSupply_scaled,TotalCoinsMined_scaled,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [278]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined_scaled", 
                       y="TotalCoinSupply_scaled", 
                       by="Class",
                       xlabel="Total Cryptocurrency Coins Mined",
                       ylabel="Total Cryptocurrency Coin Supply",)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined_scaled]   (TotalCoinSupply_scaled)